## Exercise 3: Coleman Policy Function Iteration

In [1]:
# Import packages
import numpy as np
import matplotlib.pyplot as plt
import time

# to print plots inline
%matplotlib inline

In [2]:
# Specify parameters
alpha_k = 0.29715
alpha_l = 0.65
delta = 0.154
psi = 1.08
w = 0.7
r = 0.04
sigma_eps = 0.213
mu = 0.0
rho = 0.7605
sizez = 9

betafirm = (1 / (1 + r))

In [3]:
# Use Adda-Cooper(2003) method to set up the endpoints for the z values

# import packages
from scipy.stats import norm

# Compute cut-off values
N = 9  # number of grid points (will have one more cut-off point than this)
sigma_z = sigma_eps / ((1 - rho ** 2) ** (1 / 2))
z_cutoffs = (sigma_z * norm.ppf(np.arange(N + 1) / N)) + mu
z_grid_init = ((N * sigma_z * (norm.pdf((z_cutoffs[:-1] - mu) / sigma_z)
                              - norm.pdf((z_cutoffs[1:] - mu) / sigma_z)))
              + mu)
print('Grid points = ', z_grid_init)

Grid points =  [-0.55913938 -0.32004072 -0.19425291 -0.09290094  0.          0.09290094
  0.19425291  0.32004072  0.55913938]


In [4]:
import math

"""This section of the code establishes an initial grid which will be a NumPy array
in this case of size 7x57x2."""

# Set up density grid for k and z
dens = 1
# put in bounds here for the capital stock space
grid = []

kstar = ((((1 / betafirm - 1 + delta) * ((w / alpha_l) ** (alpha_l / (1 - alpha_l)))) / (alpha_k * (1 ** (1 / (1 - alpha_l))))) **((1 - alpha_l) / (alpha_k + alpha_l - 1)))
kbar = 2*kstar
lb_k = 0.001
ub_k = kbar
krat = np.log(lb_k / ub_k)
numb = np.ceil(krat / np.log(1 - delta))
K = np.zeros(int(numb * dens))
# we'll create in a way where we pin down the upper bound - since
# the distance will be small near the lower bound, we'll miss that by little
for j in range(int(numb * dens)):
    K[j] = ub_k * (1 - delta) ** (j / dens)
kvec = K[::-1]
sizek = kvec.shape[0]

grid_star = []
lnz_grid = z_grid_init
z_grid = np.exp(z_grid_init)
for i in range(len(z_grid)):
    for j in range(sizek):
        grid_star.append(np.array([z_grid[i],kvec[j]]))
grid = np.array(grid_star)
grid = grid.reshape(N, sizek, 2)
m,n,o = np.shape(grid)

In [5]:
# Determine the transition matrix pi between each z

# import packages
import scipy.integrate as integrate

# define function that we will integrate
def integrand(x, sigma_z, sigma_eps, rho, mu, z_j, z_jp1):
    val = (np.exp((-1 * ((x - mu) ** 2)) / (2 * (sigma_z ** 2)))
            * (norm.cdf((z_jp1 - (mu * (1 - rho)) - (rho * x)) / sigma_eps)
               - norm.cdf((z_j - (mu * (1 - rho)) - (rho * x)) / sigma_eps)))
    
    return val

# compute transition probabilities
pi = np.empty((N, N))
for i in range(N):
    for j in range(N):
        results = integrate.quad(integrand, z_cutoffs[i], z_cutoffs[i + 1],
                                 args = (sigma_z, sigma_eps, rho, mu,
                                         z_cutoffs[j], z_cutoffs[j + 1]))
        pi[i,j] = (N / np.sqrt(2 * np.pi * sigma_z ** 2)) * results[0]
        
print('Transition matrix = ', pi)

Transition matrix =  [[  5.34646999e-01   2.33099908e-01   1.19241334e-01   6.16948412e-02
    3.06368282e-02   1.38694209e-02   5.27307457e-03   1.39972319e-03
    1.37871226e-04]
 [  2.33099908e-01   2.53066831e-01   1.95990939e-01   1.38221479e-01
    8.98690322e-02   5.26608516e-02   2.62024217e-02   9.48881413e-03
    1.39972319e-03]
 [  1.19241334e-01   1.95990939e-01   1.96744233e-01   1.70765394e-01
    1.33994761e-01   9.44950201e-02   5.72928238e-02   2.62024217e-02
    5.27307457e-03]
 [  6.16948412e-02   1.38221479e-01   1.70765394e-01   1.75481409e-01
    1.60693238e-01   1.32118347e-01   9.44950201e-02   5.26608516e-02
    1.38694209e-02]
 [  3.06368282e-02   8.98690322e-02   1.33994761e-01   1.60693238e-01
    1.69612281e-01   1.60693238e-01   1.33994761e-01   8.98690322e-02
    3.06368282e-02]
 [  1.38694209e-02   5.26608516e-02   9.44950201e-02   1.32118347e-01
    1.60693238e-01   1.75481409e-01   1.70765394e-01   1.38221479e-01
    6.16948412e-02]
 [  5.27307457e-03 

In [7]:
# Set up Coleman equation

def c1(k1, k, psi=1.08, delta=0.154):
    return (psi*(k1-(1-delta)*k)/k)

def c2(k1, k, psi=1.08, delta=0.154):
    return (psi / 2) * ((-2)*(k1 - (1 - delta)*k)*(1 - delta)*k - (k1 - (1 - delta)*k)**2)/(k**2)
    
def pi2(k, z, alphal=0.65, alphak=0.297, w=0.7):
    return alpha_k * (alpha_l/w) ** (alpha_l/(1 - alpha_l)) * z ** (1/(1 - alpha_l)) * k **((alpha_k - 1 + alpha_l)/(1 - alpha_l))

In [11]:
# Iterate the Euler equation
PFtol = 1e-6
PFdist = 7.0
PFmaxiter = 3000
Pinit = np.zeros((n,m))  # initial guess at policy function
P = np.zeros((n,n,m))  # initialize Pmat matrix
PFiter = 1
start_time = time.clock()

In [13]:
import numpy.linalg
from scipy.optimize import brentq

Pold = Pinit
while PFdist > PFtol and PFiter < PFmaxiter:
    #Pnew = coleman_operator(Pold)
    """Couldn't figure out exactly what would go in this part"""
    error = np.linalg.norm(Pold - Pnew)
    Pold = Pnew
    PFiter += 1
if PFiter < PFmaxiter:
    print('Policy function converged after this many iterations:', egiter)
else:
    print('Policy function did not converge')
PFI_time = time.clock() - start_time
print('PFI took ', PFI_time, ' seconds to solve')

NameError: name 'Pnew' is not defined